# Import

In [ ]:
%matplotlib inline


In [ ]:
from utils import *
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


# Running

## Setup

In [ ]:
max_evaluations = 10000  # Les critères d'arrêt 
max_iterations  = 5000

nb_runs         = 30    # le nombre d'exécution de chaque algorithme

verbose         = False # True = affichage par iteration (plus lent)

problems_name = [
    # 'knapsac',
    # 'tsp',
    # 'set_covering',
    'onemax',
    'leadingones',
    'binval'
]

#
# choix des algorithme et des paramètre 
############################################################################

algo_list = [    # Liste des algorithmes lancés (le nom des classes)
    # RandomLS,
    # HillClimbingLS,
    # FirstImprovementHillClimbingLS,
    # RandomizedHillClimbingLS,
    # SimulatedAnnealingLS,
    # TabuLS,
    # RandomGA,
    MuComaLambdaGA,
    # MuComaLambdaUniformGA,
    # MuComaLambdaRouletteGA
]
seq1 = ['one_point', 'uniform', 'one_point', 'uniform']
seq2 = ['bitflip', 'bitflip', 'uniform', 'uniform']
for i in range(0, 2):
    seq1.append(random.uniform(0.1, 0.9))
    seq2.append(random.uniform(0.1, 0.9))
algo_options_lambda = { # Paramètres spécifiques aux algorithmes. Ce dictionnaire 
    'mu' : lambda : 50,    # est passé aux constructeurs de l'algorithme 
    'lambda' : lambda : 20,
    'pc' : lambda : 0.8,
    'pm' : lambda : 0.2,
    'cdf' : lambda : 'fitness', # 'rank' ou 'fitness'
    'xover' : lambda : seq1.pop(0), # 'onepoint' ou 'uniform'
    'mutation' : lambda : seq2.pop(0), # 'bitflip' ou 'uniform'
    'selection_population' : lambda : 'deterministe', # 'aleatoire' ou 'deterministe' ou 'roulette' ou 'tournoi'
    'S' : lambda : 2,
    'alpha' : lambda : 0.1,
    'T0' : lambda : 100,
    'gamma' : lambda : 0.9,
    't': lambda : 50,
}

extra_stats = [  # Liste des chose à afficher à l'écran
    'sig',
    'max',
    'min',
    'mu',
    'alpha',
    'lambda',
    'pc',
    'pm',
    'type_cdf',
    'S',
    'gamma',
    'T',
    'T0',
    't',
    'type_xover',
    'type_mutation',
    'type_selection_population',
    'k',
    'n',

]                # (il faut surcharger la print_step dans l'agorithme)    
                    # cf. split_stat_line dans seach/misc/misc.py

## Execution

### Small problems

In [ ]:
stats = execute_and_save_stats(
    problem_size='small',
    nb_exec=1,
    max_evaluations=max_evaluations, max_iterations=max_iterations, 
    nb_runs=nb_runs, 
    algo_options_lambda=algo_options_lambda, 
    extra_stats=extra_stats, 
    verbose=verbose,
    algo_list=algo_list,
    problems_name=problems_name
)

### Medium problems

In [ ]:
stats = execute_and_save_stats(
    problem_size='medium',
    nb_exec=3,
    max_evaluations=max_evaluations, max_iterations=max_iterations, 
    nb_runs=nb_runs, 
    algo_options_lambda=algo_options_lambda, 
    extra_stats=extra_stats, 
    verbose=verbose,
    algo_list=algo_list,
    problems_name=problems_name
)

### Large problems

In [ ]:
stats = execute_and_save_stats(
    problem_size='large',
    nb_exec=3,
    max_evaluations=max_evaluations, max_iterations=max_iterations, 
    nb_runs=nb_runs, 
    algo_options_lambda=algo_options_lambda, 
    extra_stats=extra_stats, 
    verbose=verbose,
    algo_list=algo_list,
    problems_name=problems_name
)

# Plotting

## Creating the plot

In [ ]:
# Extract 'iter' and 'val' columns for plotting
# clean data (remove infinities)
data = pd.DataFrame()
stats = {}
for algo in [RandomGA, MuComaLambdaGA]:
    algo_name = algo.__name__
    for problem_name in problems_name:
        for problem_size in ['small']:
            nom_fichier = f"data/{algo_name}-{problem_name}-{problem_size}.csv"
            if  os.path.exists(nom_fichier):
                print(f"Loading {nom_fichier}")
                new_data = pd.read_csv(nom_fichier,)
                new_data['problem_size'] = problem_size
                print(new_data.keys())
                stats[algo_name] = pd.concat([new_data, stats.get(algo_name, pd.DataFrame())])

In [ ]:
data = stats['MuComaLambdaGA']
data = data[data['problem_size'] == 'small']
data

# Réponses

## Variation de mu

In [ ]:
data = stats['RandomGA']


In [ ]:
data

In [ ]:
sns.set_theme()

### Small problems

In [ ]:
data = stats['RandomGA'][stats['RandomGA']['problem_size'] == 'large']

#### Valeurs

In [ ]:
g = sns.FacetGrid(data, row="problem", col="mu", margin_titles=True,sharex='row', sharey='row')
g.map(sns.lineplot, "iter", "val", "run", alpha=.5, linewidth=0.5, palette='muted')
g.set_axis_labels("Iteration", "Valeur")
g.set_titles(col_template="{col_name} mu", row_template="{row_name} problem")
g.figure.dpi = 300
plt.suptitle('Comparaison val RandomGA - problems small')
g.tight_layout()

In [ ]:
algo_name = 'RandomGA'
x_axis_name = 'mu'
y_axis_name = 'val'
x_axis_label = 'Mu'
y_axis_label = 'Valeur max de val par run'

data = stats[algo_name]
data = data.round(2)
g = sns.FacetGrid(data.groupby(['problem', 'problem_size', x_axis_name, 'run'])[y_axis_name].max().reset_index(), row="problem", col='problem_size', margin_titles=True,sharex=False, sharey=False,height=3)
g.map(sns.boxplot, x_axis_name, y_axis_name, palette='muted')
g.set_axis_labels(x_axis_label, y_axis_label)
plt.suptitle(f'Comparaison des valeurs max de {y_axis_name} par run en fonction de {x_axis_name} - {algo_name}')
g.tight_layout()

In [ ]:
algo_name = 'MuComaLambdaGA'
x_axis_name = 'pc'
y_axis_name = 'val'
x_axis_label = 'pc'
y_axis_label = 'Valeur max de val par run'

data = stats[algo_name]
data = data.round(2)
g = sns.FacetGrid(data.groupby(['problem', 'problem_size', x_axis_name, 'run'])[y_axis_name].max().reset_index(), row="problem", col='problem_size', margin_titles=True,sharex=False, sharey=False,height=3)
g.map(sns.boxplot, x_axis_name, y_axis_name, palette='muted')
g.set_axis_labels(x_axis_label, y_axis_label)
plt.suptitle(f'Comparaison des valeurs max de {y_axis_name} par run en fonction de {x_axis_name} - {algo_name}')
g.tight_layout()

In [ ]:
algo_name = 'MuComaLambdaGA'
x_axis_name = 'pm'
y_axis_name = 'val'
x_axis_label = 'pm'
y_axis_label = 'Valeur max de val par run'

data = stats[algo_name]
data = data.round(2)
g = sns.FacetGrid(data.groupby(['problem', 'problem_size', x_axis_name, 'run'])[y_axis_name].max().reset_index(), row="problem", col='problem_size', margin_titles=True,sharex=False, sharey=False,height=3)
g.map(sns.boxplot, x_axis_name, y_axis_name, palette='muted')
g.set_axis_labels(x_axis_label, y_axis_label)
plt.suptitle(f'Comparaison des valeurs max de {y_axis_name} par run en fonction de {x_axis_name} - {algo_name}')
g.tight_layout()

#### Max

In [ ]:
g = sns.FacetGrid(data, row="problem", col="mu", margin_titles=True,sharex='row', sharey='row')
g.map(sns.lineplot, "iter", "max", "run", alpha=.5, linewidth=0.5, palette='muted')
g.set_axis_labels("Iteration", "Max")
g.set_titles(col_template="{col_name} mu", row_template="{row_name} problem")
g.figure.dpi = 300
plt.suptitle('Comparaison max RandomGA - problems small')
g.tight_layout()

#### Min

In [ ]:
g = sns.FacetGrid(data, row="problem", col="mu", margin_titles=True,sharex='row', sharey='row')
g.map(sns.lineplot, "iter", "min", "run", alpha=.5, linewidth=0.5, palette='muted')
g.set_axis_labels("Iteration", "Min")
g.set_titles(col_template="{col_name} mu", row_template="{row_name} problem")
g.figure.dpi = 300
plt.suptitle('Comparaison min RandomGA - problems small')
g.tight_layout()

## Variation de pc et pm

In [ ]:
data = stats['MuComaLambdaGA']


In [ ]:
data

### Small problems

In [ ]:
data = stats['MuComaLambdaGA'][stats['MuComaLambdaGA']['problem_size'] == 'small']

#### Valeurs

In [ ]:
g = sns.FacetGrid(data, row="problem", col="pc", margin_titles=True,sharex='row', sharey='row')
g.map(sns.lineplot, "iter", "val", "run", alpha=.5, linewidth=0.5, palette='muted')
g.set_axis_labels("Iteration", "Valeur")
g.set_titles(col_template="pc: {col_name}", row_template="{row_name} problem")
g.figure.dpi = 300
plt.suptitle('Comparaison val MuComaLambdaGA - problems small')
g.tight_layout()

In [ ]:
g = sns.FacetGrid(data, row="problem", col="pm", margin_titles=True,sharex='row', sharey='row')
g.map(sns.lineplot, "iter", "val", "run", alpha=.5, linewidth=0.5, palette='muted')
g.set_axis_labels("Iteration", "Valeur")
g.set_titles(col_template="pm: {col_name}", row_template="{row_name} problem")
g.figure.dpi = 300
plt.suptitle('Comparaison val MuComaLambdaGA - problems small')
g.tight_layout()

#### Max

In [ ]:
g = sns.FacetGrid(data, row="problem", col="pc", margin_titles=True,sharex='row', sharey='row')
g.map(sns.lineplot, "iter", "max", "run", alpha=.5, linewidth=0.5, palette='muted')
g.set_axis_labels("Iteration", "Valeur")
g.set_titles(col_template="pc: {col_name}", row_template="{row_name} problem")
g.figure.dpi = 300
plt.suptitle('Comparaison val MuComaLambdaGA - problems small')
g.tight_layout()

In [ ]:
g = sns.FacetGrid(data, row="problem", col="pm", margin_titles=True,sharex='row', sharey='row')
g.map(sns.lineplot, "iter", "max", "run", alpha=.5, linewidth=0.5, palette='muted')
g.set_axis_labels("Iteration", "Valeur")
g.set_titles(col_template="pm: {col_name:.2f }", row_template="{row_name} problem")
g.figure.dpi = 300
plt.suptitle('Comparaison val MuComaLambdaGA - problems small')
g.tight_layout()

#### Min

In [ ]:
g = sns.FacetGrid(data, row="problem", col="mu", margin_titles=True,sharex='row', sharey='row')
g.map(sns.lineplot, "iter", "min", "run", alpha=.5, linewidth=0.5, palette='muted')
g.set_axis_labels("Iteration", "Valeur")
g.set_titles(col_template="{col_name} mu", row_template="{row_name} problem")
g.figure.dpi = 300
plt.suptitle('Comparaison min RandomGA - problems small')
g.tight_layout()